# Optimizing Financial Portfolios with Reinforcement Learning: Data retrival and processing

This notebook serves as a foundational component of the thesis, focusing on the meticulous analysis and preparation of financial data crucial for portfolio optimization using reinforcement learning. It encompasses the following key aspects:

- Stock selection
- Data collection through Alpha advantage API + EODHD Financal News & Sentiment API
- Data Processing + Feature enginnerring
- Data Splitting into test + train datasets


## Stock selection

Creating a balanced portfolio typically involves selecting a diverse set of stocks from various sectors and industries to mitigate risk. The idea is to include stocks that are less likely to be affected by the same economic factors at the same time. Here's a list of 20 stocks from different sectors that could represent a balanced portfolio. Note that the focus of this reseach is on the rebalancing of a portfolio, not on the selection of stocks.

### Technology
1. **Apple Inc. (AAPL)** - A leader in consumer electronics and technology.
2. **Microsoft Corporation (MSFT)** - A dominant player in software and cloud services.
3. **NVIDIA Corporation (NVDA)** - Specializes in graphics processing units (GPUs).
4. **Adobe Inc. (ADBE)**: A software company known for its creative and multimedia software products.


### Healthcare
4. **Johnson & Johnson (JNJ)** - A well-established company in pharmaceuticals and consumer goods.
5. **Pfizer Inc. (PFE)** - A global biopharmaceutical company.

### Financials
7. **JPMorgan Chase & Co. (JPM)** - A leading global financial services firm.
8. **Berkshire Hathaway Inc. (BRK.B)**: A well-established diversified holding company.

### Consumer Discretionary
10. **Amazon.com, Inc. (AMZN)** - A giant in e-commerce and cloud computing.
11. **Tesla, Inc. (TSLA)** - Renowned for its electric vehicles and clean energy.
12. **Nike, Inc. (NKE)** - A major manufacturer of athletic footwear and apparel.
13. **The Walt Disney Company (DIS)**: A diversified international entertainment company with operations in media networks, parks and resorts, studio entertainment, and consumer products.

### Consumer Staples
13. **Costco Wholesale Corporation (COST)** - Operates membership warehouses offering a variety of products.
15. **Walmart Inc. (WMT)** - The world's largest retailer.

### Industrials
16. **3M Company (MMM)** - Known for its diversified technology.
17. **The Boeing Company (BA)** - A major aerospace and defense company.
18. **Waste Management, Inc. (WM)**: North America's leading provider of integrated environmental solutions, managing waste and recycling services.

### Energy
19. **Pacific Gas and Electric Company (PCG)**: One of the largest combined natural gas and electric energy companies in the United States, serving Northern and Central California.
20. **NextEra Energy, Inc. (NEE)** - A leader in renewable energy.

### Considerations:
- **Geographical Diversification**: Depending on your risk tolerance and investment goals, consider adding international stocks for geographical diversification.
- **Sector Balance**: Regularly review and balance the sectors in your portfolio to ensure that no single sector dominates.
- **Market Capitalization**: Including a mix of large-cap, mid-cap, and small-cap stocks can provide further diversification.
- **Periodic Rebalancing**: The market changes, so it's crucial to periodically rebalance your portfolio to maintain your desired level of diversification and risk.
- **Current Market Trends**: Stay informed about current market trends and economic factors that could impact these sectors.

Remember, the right mix of stocks for a balanced portfolio can vary greatly depending on individual financial goals, risk tolerance, and investment horizon.


In [28]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

apikey = 'TKBT5GP25LDEHY61'
sym = 'AAPL'
train_window = 9
test_window = 1

In [40]:
# Define the list of ticker symbols
ticker_symbols = [
    "AAPL",
    "MSFT",
    "NVDA",
    "ADBE",
    "JNJ",
    "PFE",
    "JPM",
    "BRK.B",
    "AMZN",
    "TSLA",
    "NKE",
    "DIS",
    "COST",
    "WMT",
    "MMM",
    "BA",
    "WM",
    "PCG",
    "NEE" 
]

# Create an empty DataFrame to store the merged data
merged_portfolio_df = pd.DataFrame()

# Specify train and test windows
train_window = 9
test_window = 1

# Loop through each ticker symbol
for sym in ticker_symbols:
    #Daily prices + volumes
    url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol='+sym+'&outputsize=full&apikey='+apikey
    r = requests.get(url)
    data = r.json()
    df = pd.DataFrame(data['Time Series (Daily)']).T
    df.index = pd.to_datetime(df.index)  
    # Renaming the columns as per the provided structure
    df.columns = ['open', 'high', 'low', 'close', 'adjusted close', 'volume', 'dividend amount', 'split coefficient']
    # Convert all data to float, except the date which should be the index
    df = df.astype({'open': 'float', 'high': 'float', 'low': 'float', 'close': 'float', 
                    'adjusted close': 'float', 'volume': 'float', 'dividend amount': 'float', 
                    'split coefficient': 'float'})


    # Calculate Turbulance
    df.sort_index(ascending=True, inplace=True)
    #for col in ['open', 'high', 'low', 'close', 'volume']:
     #   df[col] = pd.to_numeric(df[col], errors='coerce')
    df['daily gain'] = df['close'].pct_change()
    df['turbulence'] = df['daily gain'].rolling(window=20).std()
    # Calculate SMA50 and SMA20
    df['ma50'] = df['close'].rolling(window=50).mean()
    df['ma20'] = df['close'].rolling(window=20).mean()
    # Calculate EMA50 and EMA20
    df['ema20'] = df['close'].ewm(span=20, adjust=False).mean()
    df['ema50'] = df['close'].ewm(span=50, adjust=False).mean()
    df.sort_index(ascending=False, inplace=True)


    # Calculate volume change
    df['volume change'] = df['volume'].pct_change()

    #RSI
    url = 'https://www.alphavantage.co/query?function=RSI&symbol='+sym+'&interval=daily&time_period=10&series_type=open&apikey='+apikey
    r = requests.get(url)
    data = r.json()
    RSI = pd.DataFrame(data['Technical Analysis: RSI']).T
    RSI.index = pd.to_datetime(RSI.index)  


    #BBands
    url = 'https://www.alphavantage.co/query?function=BBANDS&symbol='+sym+'&interval=daily&time_period=50&series_type=close&nbdevup=3&nbdevdn=3&apikey='+apikey
    r = requests.get(url)
    bbands = r.json()
    bbands = bbands['Technical Analysis: BBANDS']
    bbands = pd.DataFrame(bbands).T  
    bbands.index = pd.to_datetime(bbands.index)  

    #MACD
    url = 'https://www.alphavantage.co/query?function=MACD&symbol='+sym+'&interval=daily&series_type=open&apikey='+apikey
    r = requests.get(url)
    data = r.json()
    data = data['Technical Analysis: MACD']
    macd = pd.DataFrame(data).T  
    macd.index = pd.to_datetime(macd.index)

    # STOCH
    url = 'https://www.alphavantage.co/query?function=STOCH&symbol='+sym+'&interval=daily&apikey='+apikey
    r = requests.get(url)
    stoch = r.json()
    stoch = stoch['Technical Analysis: STOCH']
    stoch = pd.DataFrame(stoch).T  
    stoch.index = pd.to_datetime(stoch.index)  
    
    # AROON
    url = 'https://www.alphavantage.co/query?function=AROON&symbol='+sym+'&interval=daily&time_period=25&apikey='+apikey
    r = requests.get(url)
    aroon = r.json()
    aroon = aroon['Technical Analysis: AROON']
    aroon = pd.DataFrame(aroon).T  
    aroon.index = pd.to_datetime(aroon.index)  

    # OBV
    url = 'https://www.alphavantage.co/query?function=OBV&symbol='+sym+'&interval=daily&apikey='+apikey
    r = requests.get(url)
    obv = r.json()
    obv = obv['Technical Analysis: OBV']
    obv = pd.DataFrame(obv).T  
    obv.index = pd.to_datetime(obv.index)  

    # adx
    url = 'https://www.alphavantage.co/query?function=ADX&symbol='+sym+'&interval=daily&time_period=10&apikey='+apikey
    r = requests.get(url)
    adx = r.json()
    adx = adx['Technical Analysis: ADX']
    adx = pd.DataFrame(adx).T  
    adx.index = pd.to_datetime(adx.index)  
    
    # Chaikin A/D
    url = 'https://www.alphavantage.co/query?function=AD&symbol='+sym+'&interval=daily&apikey='+apikey
    r = requests.get(url)
    ad = r.json()
    ad = ad['Technical Analysis: Chaikin A/D']
    ad = pd.DataFrame(ad).T  
    ad.index = pd.to_datetime(ad.index)  
    
    # CCI
    url = 'https://www.alphavantage.co/query?function=CCI&symbol='+sym+'&interval=daily&time_period=20&apikey='+apikey
    r = requests.get(url)
    cci = r.json()
    cci = cci['Technical Analysis: CCI']
    cci = pd.DataFrame(cci).T  
    cci.index = pd.to_datetime(cci.index)  

    # Filter data based on train + test window
    earliest_date = df.index.min() 
    time_window = datetime.now() - timedelta(days=365*(train_window+test_window))
    if time_window < earliest_date:
        raise ValueError(f"The specified time window is before the earliest available date for '{sym}'.")
    time_window = datetime.now() - timedelta(days=365*(train_window+test_window))
    df = df[df.index > time_window]

    split_indices = df[df['split coefficient'] != 1].index
    for index in split_indices:
        split_factor = df.loc[index, 'split coefficient']
        df.loc[:index, ['open', 'high', 'low', 'close', 'adjusted close']] *= split_factor
    df.index = pd.to_datetime(df.index)  
    df = df.drop(['dividend amount', 'adjusted close', 'split coefficient'], axis=1)

    RSI = RSI[RSI.index > time_window]
    bbands = bbands[bbands.index > time_window]
    macd = macd[macd.index > time_window]

    # Merge all DataFrames on the date index
    merged_df = df.join(RSI, how='inner').join(bbands, how='inner').join(macd, how='inner').join(stoch, how='inner').join(aroon, how='inner').join(obv, how='inner').join(adx, how='inner').join(ad, how='inner').join(cci, how='inner')
    # Resetting index to turn the date index into a column
    merged_df = merged_df.reset_index()
    # Renaming the index column to 'Date'
    merged_df.rename(columns={'index': 'Date'}, inplace=True)
    # Adding a ticker column
    merged_df['tic'] = sym
    # Rearranging columns starting with date and tic
    merged_df = merged_df[['Date','tic'] + [col for col in merged_df.columns if col not in ['Date','tic']]]
    # Merge the data for the current symbol into the merged_portfolio_df
    merged_portfolio_df = pd.concat([merged_portfolio_df, merged_df])
    # Delay API calls to keep under limit
    time.sleep(1)
    print(sym)


AAPL
MSFT
NVDA
ADBE
JNJ
PFE
JPM
BRK.B
AMZN
TSLA
NKE
DIS
COST
WMT
MMM
BA
WM
PCG
NEE


In [42]:
# Fetching data for SPY (S&P 500 ETF)
sym = "SPY"
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={sym}&outputsize=full&apikey={apikey}'
r = requests.get(url)
data = r.json()

# Preparing the DataFrame
spy = pd.DataFrame(data['Time Series (Daily)']).T
spy.index = pd.to_datetime(spy.index)
spy.sort_index(inplace=True)
spy['4. close'] = spy['4. close'].astype(float)
spy['SP500_delta'] = spy['4. close'].pct_change() * 100
spy.sort_index(ascending=False, inplace=True)
spy = pd.DataFrame(spy['SP500_delta'])

# Merging the DataFrames
merged_portfolio_df = pd.merge(merged_portfolio_df, spy, left_on='Date', right_index=True, how='left')

merged_portfolio_df


,Date,tic,open,high,low,close,volume,daily gain,turbulence,ma50,...,MACD_Hist,SlowK,SlowD,Aroon Down,Aroon Up,OBV,ADX,Chaikin A/D,CCI,SP500_delta
0,2023-12-29,AAPL,5429.205429,5443.205443,5368.305368,5390.845391,42672148.0,-0.005424,0.009078,186.6338,...,-0.7621,35.3874,25.8554,28.0000,60.0000,95729905414.0000,23.5891,80479696956.0885,-48.7932,-0.289496
1,2023-12-28,AAPL,5435.925436,5450.485450,5408.765409,5420.245420,34049898.0,0.002226,0.008973,186.3000,...,-0.7552,25.7883,18.7256,32.0000,64.0000,95772577562.0000,24.6667,80496686054.2642,-6.8407,0.037775
2,2023-12-27,AAPL,5389.725390,5418.005418,5350.525351,5408.205408,47899806.0,0.000518,0.009086,185.9714,...,-0.7471,16.3905,15.2263,36.0000,68.0000,95738527664.0000,27.1271,80511997082.2240,-39.4339,0.180805
3,2023-12-26,AAPL,5421.085421,5428.925429,5399.245399,5405.405405,28919310.0,-0.002841,0.009103,185.6828,...,-0.5481,13.9980,18.7908,40.0000,72.0000,95690627858.0000,28.6599,80478010082.9460,-13.4523,0.422253
4,2023-12-22,AAPL,5465.045465,5471.485471,5403.165403,5420.805421,37149570.0,-0.005548,0.009071,185.3988,...,-0.3458,15.2903,26.3414,44.0000,76.0000,95719547168.0000,31.7276,80494925151.0592,12.9401,0.200973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,2014-01-09,NEE,85.570000,86.210000,84.920000,86.020000,2067300.0,0.008559,0.006412,85.2970,...,0.0284,73.9416,52.7918,44.0000,100.0000,1442706956.0000,17.9141,1082392155.8866,117.3357,0.065388
2511,2014-01-08,NEE,85.060000,85.750000,84.480000,85.290000,3086300.0,0.003176,0.007248,85.2798,...,0.0221,51.1006,33.2226,48.0000,80.0000,1434437756.0000,17.2236,1076558844.2587,86.2444,0.021801
2512,2014-01-07,NEE,84.600000,85.325000,84.500000,85.020000,1842200.0,0.007346,0.007380,85.2930,...,0.0201,33.3333,28.4577,52.0000,84.0000,1422092556.0000,17.4758,1073156623.7862,76.7893,0.614170
2513,2014-01-06,NEE,84.430000,84.640000,84.000000,84.400000,1690900.0,0.000474,0.007718,85.2960,...,0.0244,15.2340,35.7153,56.0000,88.0000,1414723756.0000,18.8125,1071236269.8468,31.2402,-0.289792


In [43]:
maxdate=merged_portfolio_df['Date'].max().date()
mindate=merged_portfolio_df['Date'].min().date()

url = f'https://eodhd.com/api/sentiments?s=aapl,msft,nvda,adbe,jnj,pfe,jpm,brk.b,amzn,tsla,nke,dis,cost,wmt,mmm,ba,wm,pcg,nee&from={mindate}&to={maxdate}&api_token=6586f89ee51fc6.48205259&fmt=json'
data = requests.get(url).json()

sentiment_data = []
for tic, records in data.items():
    for record in records:
        record['tic'] = tic.split('.')[0]  # Extract ticker symbol and remove market suffix if any
        sentiment_data.append(record)

sentiment_df = pd.DataFrame(sentiment_data)
sentiment_df['date'] = pd.to_datetime(sentiment_df['date'])
sentiment_df = sentiment_df.rename(columns={'date':'Date','count': 'count articles', 'normalized': 'sentiment value'})

# Merging the DataFrames on 'date' and 'tic'
merged_portfolio_df = pd.merge(merged_portfolio_df, sentiment_df, on=['Date', 'tic'], how='left')


In [45]:
#Cleaning & process data

#merged_portfolio_df = merged_portfolio_df

# Fill missing values with the previous value
merged_portfolio_df.fillna(method='backfill', inplace=True)
merged_portfolio_df['count articles'] = merged_portfolio_df['count articles'].fillna(0)
merged_portfolio_df['sentiment value'] = merged_portfolio_df['sentiment value'].fillna(0)


# Convert columns to numeric, excluding 'Ticker' and 'Date'
for col in [c for c in merged_portfolio_df.columns if c not in ['tic', 'Date']]:
    merged_portfolio_df[col] = pd.to_numeric(merged_portfolio_df[col], errors='coerce')

    
# Add Calculated feilds
merged_portfolio_df['close-ma50'] = merged_portfolio_df['close'] - merged_portfolio_df['ma50']
merged_portfolio_df['close-ma20'] = merged_portfolio_df['close'] - merged_portfolio_df['ma20']
merged_portfolio_df['close-ema20'] = merged_portfolio_df['close'] - merged_portfolio_df['ema50']
merged_portfolio_df['close-ema50'] = merged_portfolio_df['close'] - merged_portfolio_df['ema50']
merged_portfolio_df['close-upperbb'] = merged_portfolio_df['close'] - merged_portfolio_df['Real Upper Band']
merged_portfolio_df['close-middlebb'] = merged_portfolio_df['close'] - merged_portfolio_df['Real Middle Band']
merged_portfolio_df['close-lowerbb'] = merged_portfolio_df['close'] - merged_portfolio_df['Real Lower Band']
merged_portfolio_df['day'] = merged_portfolio_df['Date'].dt.dayofweek
# Sort the DataFrame by date and ticker
merged_portfolio_df = merged_portfolio_df.sort_values(by=['Date', 'tic'],ascending=[False, True])

# Convert Column names to lower
merged_portfolio_df.columns = merged_portfolio_df.columns.str.lower()

# Reset the index after sorting
merged_portfolio_df = merged_portfolio_df.sort_values(by=['date', 'tic'],ascending=[False, True])
merged_portfolio_df = merged_portfolio_df.reset_index(drop=True)


# Split into test and train dataframs
test_p_df = merged_portfolio_df[merged_portfolio_df.index < (365*(test_window)*len(ticker_symbols))]
train_p_df = merged_portfolio_df[merged_portfolio_df.index >= (365*(test_window)*len(ticker_symbols))]

# Reset index for each ticker
unique_tickers = train_p_df['tic'].unique()
train_p_df = train_p_df.sort_values(by=['date', 'tic'],ascending=[True, True])
df_list = []
for ticker in unique_tickers:
    df_ticker = train_p_df[train_p_df['tic'] == ticker].reset_index(drop=True)
    df_list.append(df_ticker)
merged_train_p_df = pd.concat(df_list, axis=0)
merged_train_p_df = merged_train_p_df.sort_values(by=['date', 'tic'],ascending=[True, True])
train_p_df = merged_train_p_df

# Reset index for each ticker
test_p_df = test_p_df.sort_values(by=['date', 'tic'],ascending=[True, True])
df_list = []
for ticker in unique_tickers:
    df_ticker = test_p_df[test_p_df['tic'] == ticker].reset_index(drop=True)
    df_list.append(df_ticker)
merged_test_p_df = pd.concat(df_list, axis=0)
merged_test_p_df = merged_test_p_df.sort_values(by=['date', 'tic'],ascending=[True, True])
test_p_df = merged_test_p_df


In [46]:
test_p_df.to_csv('data/test_p_df.csv', index=True)  
train_p_df.to_csv('data/train_p_df.csv', index=True)  